In [10]:
print('importing')

try:
    import pandas as pd
    from datasets import load_dataset
    from PIL import Image
    import numpy as np
    import time
    import openai
    import os
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    import re
    from tqdm import tqdm
    from collections import OrderedDict
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import accuracy_score, confusion_matrix
    import matplotlib.pyplot as plt
    from sklearn.naive_bayes import GaussianNB
    from imblearn.over_sampling import SMOTE
    smote = SMOTE()
    print('Import complete')
except ImportError as e:
    print(f"Error importing module: {e}")

# Set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
import getpass
if openai.api_key is None:
    print(' ValueError("API key is missing. Please set the OPENAI_API_KEY environment variable.")')
    openai.api_key = getpass.getpass("Enter your API key: ")

# Initialize the OpenAI client
client = openai
try:
    print('Loading Dataset...')
    prompts = pd.read_csv('df_subset.csv', 
    usecols=['Numbers', 'allowed'],
    low_memory=False
)
    df=pd.read_csv('WordList.csv')
except FileNotFoundError as e:
    print(f"Error loading dataset: {e}")



def classify_word(word):
    # Define a system message and user prompt to classify each word
    prompt = f"Classify the word '{word}' into one of the following categories: 'illegal', 'explicit', 'violent', 'innocent'. Provide just the category as the output. Ensure one of these categories is no matter what stated."

    # Call the OpenAI API with the prompt
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        
        # Parse the response and extract the classification
        classification = response.choices[0].message.content.strip()
        if 'illegal' in classification:
            classification = 'illegal'
        elif 'explicit' in classification:
            classification = 'explicit'
        elif 'violent' in classification:
            classification = 'violent'
        elif 'innocent' in classification:
            classification = 'innocent'
        else:
            response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=30
        )
        # Map the response to a unique value
        category_mapping = {
            "illegal": 1,
            "explicit": 2,
            "violent": 3,
            "innocent": 0
        }
        
        # Return the mapped value
        return category_mapping.get(classification, -1)  # Return -1 for unknown categories
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1
    
def numberify(word):
    # Define the user prompt
    prompt = f"HERE IS THE WORD: {word}"

    try:
        # First classification: 4-digit number
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """Assign the word a number based on the following criteria.
    The number will be 4 digits. The first 4 digits determine how 'bad' the word may be.
    - Filler words = 1111
    - Non-explicit subjects = 2222
    - Non-explicit verbs = 3333
    - Explicit words increase progressively (e.g., 5555, 6666, 7777, 8888)
    - Use 9999 only for extreme cases.
    Respond with just the number!."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        classification = response.choices[0].message.content.strip()

        # Second classification: 2-digit number based on implication
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": """Assign the word a number from 00 to 99 based on severity.
    - Use a balanced scale with 99 reserved for extreme words.
    Respond with just the number!."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        secondary = response.choices[0].message.content.strip()

        # Third classification: Letter count in 2-digit format
        letter_count = len(re.findall(r'[a-zA-Z]', word))
        formatted_count = str(letter_count).zfill(2)  # Ensures 2-digit format

        # Combine all values into a single 8-digit number
        final_number = int(f"{classification}{formatted_count}{secondary}")


        return final_number, word  # Returns the combined number and word
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1, word  # Returns -1 on error
df['Word'] = df['Word'].str.lower()

print("Training models...")
X = prompts['Numbers'].apply(lambda x: sum(x) if isinstance(x, list) else 0).values.reshape(-1, 1)  # If 'Numbers' is a list, sum them as a feature
y = prompts['allowed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
nb = GaussianNB(var_smoothing=1e-8)
nb.fit(X_train_resampled, y_train_resampled)
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)
print("Model training complete.")

while True:
    custom_prompt = input("Enter your prompt (or 'exit' to quit): ")
    if custom_prompt.lower() == 'exit':
        break
    elif custom_prompt.strip() == '':
        print("Prompt cannot be empty.")
        continue
    rows = []
    Classprompt = custom_prompt.lower()
    word_classifications = df.set_index('Word').to_dict()
            # Clean up non-alphabetical characters (remove special characters)
    Classprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  
    Numprompt = re.sub(r'[^a-zA-Z\s]', '', Classprompt)  

    numeric_values = []
    classification_values = []

    # Process words and classify them
    for word in re.findall(r'\b\w+\b', Classprompt):  # Loop through each word
        word_row = df[df['Word'] == word]
        if word_row.empty:
            # Here, classify the word based on your own logic, e.g., classify_word, numberify
            classification_value = classify_word(word)  # Classify word (you'll have to define this)
            numeric_value, interpreted_word = numberify(word)  # Interpret numeric value (define this too)
            new_row = pd.DataFrame({'Word': [word], 'Classification': [classification_value], 
                                    'Number': [numeric_value]})
            df = pd.concat([df, new_row], ignore_index=True)
            
            # Use the newly classified values
            current_classification = classification_value
            current_numeric_value = numeric_value
        else:
            current_classification = word_row['Classification'].values[0]
            current_numeric_value = word_row['Number'].values[0]
            print(current_classification)
            print(current_numeric_value)
        Classprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(current_classification), Classprompt)
        Numprompt = re.sub(r'\b' + re.escape(word) + r'\b', str(current_numeric_value), Numprompt)
    
    print("Processed Classprompt:", Classprompt)
    print("Processed Numprompt:", Numprompt)
    print("Pre Checks")
    Class_3_count = Classprompt.count('3')
    Class_2_count = Classprompt.count('2')
    Class_1_count = Classprompt.count('1')

    Class_Sum_count = Class_3_count + Class_2_count + Class_1_count

    # Apply filtering conditions
    Guess_allowed = 1  # Default assumption (if conditions are not violated)
    if Class_3_count > 4 or Class_2_count > 4 or Class_1_count > 4 or Class_Sum_count > 4:
        Guess_allowed = 0

    # Extract numbers from Numprompt
    Numbers = list(map(int, re.findall(r'\d+', Numprompt)))
    Average_Number = sum(Numbers)/len(Numbers) if Numbers else 0

    if Average_Number > 50000000:
        Guess_allowed = 0
    elif any(num > 60000000 for num in Numbers):
        Guess_allowed = 0

    # Print the result for the custom prompt
    print(f"Filter check allowed: {Guess_allowed}")

    print("running model...")
    def preprocess_custom_prompt(prompt):
        # Extract numbers from the prompt
        numbers = re.findall(r'\d+', prompt)
    
        # Convert the numbers to integers
        numbers = list(map(int, numbers))
        
        # Sum the numbers (or use alternative feature engineering)
        sum_of_numbers = sum(numbers) if numbers else 0
        
        # Ensure non-zero variance and prevent extreme values
        sum_of_numbers = max(1, min(sum_of_numbers, 1e8))
        
        # Reshape to match model input requirements
        return np.array([[sum_of_numbers]])

    X_custom = preprocess_custom_prompt(Numprompt)
    custom_prediction = knn.predict(X_custom)
    print(f"KNN Prediction for the custom prompt: {custom_prediction[0]}")
    NBPredict = nb.predict(X_custom)
    print(f"Naive Bayes Prediction for the custom prompt: {NBPredict[0]}")

    

    

importing
Import complete
Loading Dataset...
Training models...
Model training complete.


Enter your prompt (or 'exit' to quit):  children looking happily at starry night


C:\Users\georg\anaconda3\Lib\site-packages\sklearn\naive_bayes.py:510: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
C:\Users\georg\anaconda3\Lib\site-packages\sklearn\naive_bayes.py:511: RuntimeWarning: divide by zero encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
C:\Users\georg\anaconda3\Lib\site-packages\sklearn\naive_bayes.py:511: RuntimeWarning: invalid value encountered in subtract
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)


0
22220810
0
33330720
0
33330705
0
11110200
0
22220610
0
22220510
Processed Classprompt: 0 0 0 0 0 0
Processed Numprompt: 22220810 33330720 33330705 11110200 22220610 22220510
Pre Checks
Filter check allowed: 1
running model...
KNN Prediction for the custom prompt: 1
Naive Bayes Prediction for the custom prompt: 0


KeyboardInterrupt: Interrupted by user

In [8]:
prompts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867760 entries, 0 to 867759
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   image                     50000 non-null   object 
 1   prompt                    867578 non-null  object 
 2   seed                      50000 non-null   float64
 3   step                      50000 non-null   float64
 4   cfg                       50000 non-null   float64
 5   sampler                   50000 non-null   object 
 6   width                     50000 non-null   float64
 7   height                    50000 non-null   float64
 8   user_name                 50000 non-null   object 
 9   timestamp                 50000 non-null   object 
 10  image_nsfw                867760 non-null  float64
 11  prompt_nsfw               867760 non-null  float64
 12  allowed                   867760 non-null  int64  
 13  Classprompt               867213 non-null  o

In [ ]:
df.head()

In [9]:
print(prompts.dtypes)
print(prompts.iloc[:, [0,5,8,9]].dtypes)

image                        object
prompt                       object
seed                        float64
step                        float64
cfg                         float64
sampler                      object
width                       float64
height                      float64
user_name                    object
timestamp                    object
image_nsfw                  float64
prompt_nsfw                 float64
allowed                       int64
Classprompt                  object
Numprompt                    object
Class 3 appearance count    float64
Class 2 appearance count    float64
Class 1 appearance count    float64
Class Sum count             float64
Guess allowed               float64
Numbers                      object
Average Number              float64
Banned                        int64
dtype: object
image        object
sampler      object
user_name    object
timestamp    object
dtype: object
